<a href="https://colab.research.google.com/github/Adarsh-Vemali/Natural-Language-Processing/blob/master/Passage_comprehension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question Answering with a Fine-Tuned BERT
*by Chris McCormick*

In [1]:
!pip install transformers

     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 3.0MB 12.2MB/s 
     |████████████████████████████████| 1.1MB 31.7MB/s 
     |████████████████████████████████| 890kB 24.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=05770007657bc80fe4339ff2a07e59e9548e62c7d88d620e16dceab283a0b412
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import torch

In [3]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


In [5]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [6]:
question = "what was the ambition?"
answer_text = "(1) I had ambition not only to go further than any man had been before; but as far as it was possible for man to go, wrote James Cook, the explorer, who added Australia and New Zealand to the BritishEmpire.(ii) James Cook, the father of Antarctic exploration, was born in Marton village, Cleveland, on October 28, 1728. From his boyhood, he was interested in seafaring. One day the lad made up his mind; he, too, was going to sea in order to visit glamorous lands. At the age of twenty-seven, Cook had risen to the position of first mate. The first service Cook saw was in Canada, where he was employed in the dangerous task of surveying the St. Lawrence Island. (iii) When Cook, on August 25, 1768, with a company of eighty-three men (including a party of scientists, among whom was the great Sir Joseph Banks) set sail in the Endeavour, they had before them the possibility of filling in a substantial area of the globe's surface. They reached Tahiti in the spring of 1769. Cook sailed south on his quest for the unknown continent, and skirting the Society Islands, at length reached New Zealand. The tattooed natives met them. Cook greeted these Maori warriors with friendly signs and eventually prevailed on them to lay down their spears in sign of truce. After circumnavigating the North and South Islands, Cook surveyed the coastline and landed at Queen Charlotte's Sound. He then hoisted the Union Jack and informed his company that he had taken possession of the islands on behalf of His Majesty George the Third."

We'll need to run the BERT tokenizer against both the `question` and the `answer_text`. To feed these into BERT, we actually concatenate them together and place the special [SEP] token in between.


In [7]:
# Apply the tokenizer to the input text, treating them as a text-pair.
input_ids = tokenizer.encode(question, answer_text)

print('The input has a total of {:} tokens.'.format(len(input_ids)))

The input has a total of 343 tokens.


In [8]:
# BERT only needs the token IDs, but for the purpose of inspecting the 
# tokenizer's behavior, let's also get the token strings and display them.
tokens = tokenizer.convert_ids_to_tokens(input_ids)

# For each token and its id...
for token, id in zip(tokens, input_ids):
    
    # If this is the [SEP] token, add some space around it to make it stand out.
    if id == tokenizer.sep_token_id:
        print('')
    
    # Print the token string and its ID in two columns.
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')
    

[CLS]           101
what          2,054
was           2,001
the           1,996
ambition     16,290
?             1,029

[SEP]           102

(             1,006
1             1,015
)             1,007
i             1,045
had           2,018
ambition     16,290
not           2,025
only          2,069
to            2,000
go            2,175
further       2,582
than          2,084
any           2,151
man           2,158
had           2,018
been          2,042
before        2,077
;             1,025
but           2,021
as            2,004
far           2,521
as            2,004
it            2,009
was           2,001
possible      2,825
for           2,005
man           2,158
to            2,000
go            2,175
,             1,010
wrote         2,626
james         2,508
cook          5,660
,             1,010
the           1,996
explorer     10,566
,             1,010
who           2,040
added         2,794
australia     2,660
and           1,998
new           2,047
zealand       3,41

In [9]:
# Search the input_ids for the first instance of the `[SEP]` token.
sep_index = input_ids.index(tokenizer.sep_token_id)

# The number of segment A tokens includes the [SEP] token istelf.
num_seg_a = sep_index + 1

# The remainder are segment B.
num_seg_b = len(input_ids) - num_seg_a

# Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b

# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)

In [10]:
# Run our example through the model.
start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                 token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text


In [11]:
# Find the tokens with the highest `start` and `end` scores.
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

# Combine the tokens in the answer and print it out.
answer = ' '.join(tokens[answer_start:answer_end+1])

print('Answer: "' + answer + '"')

Answer: "to go further than any man had been before ; but as far as it was possible for man to go"


In [12]:
# Start with the first token.
answer = tokens[answer_start]

# Select the remaining answer tokens and join them with whitespace.
for i in range(answer_start + 1, answer_end + 1):
    
    # If it's a subword token, then recombine it with the previous token.
    if tokens[i][0:2] == '##':
        answer += tokens[i][2:]
    
    # Otherwise, add a space then the token.
    else:
        answer += ' ' + tokens[i]

print('Answer: "' + answer + '"')

Answer: "to go further than any man had been before ; but as far as it was possible for man to go"
